In [1]:
import unetgee as ug
import ee
from datetime import datetime
from dateutil.relativedelta import relativedelta

2024-08-14 04:43:22.835764: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 04:43:22.993437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 04:43:26.406249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 04:43:26.424337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bu

In [2]:
ee.Authenticate(force=True)
ee.Initialize()


Successfully saved authorization token.


In [3]:
hybas_gt100_id_list = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroBASINS_lev02/hybas_lev02_v1c_merged_lake_count_added_gt100").aggregate_array("HYBAS_ID").getInfo()
print(hybas_gt100_id_list)

[1020000010, 8020000010, 4020000010, 5020000010, 2020000010, 9020000010, 7020000010, 6020000010, 3020000010, 1020011530, 8020008900, 4020006940, 5020015660, 2020003440, 7020014250, 6020006540, 3020003790, 1020018110, 8020010700, 4020015090, 5020037270, 2020018240, 7020021430, 6020008320, 3020005240, 1020021940, 8020020760, 4020024190, 5020049720, 2020024230, 7020024600, 6020014330, 3020008670, 1020027430, 8020022890, 4020034510, 2020033490, 7020038340, 6020017370, 3020009320, 1020034170, 8020032840, 2020041390, 7020046750, 6020021870, 1020035180, 8020044560, 5020082270, 2020057170, 7020047840, 6020029280, 2020065840, 7020065090, 2020071190, 4020050210, 3020024310, 4020050220, 1020040190, 4020050290, 4020050470]


In [3]:
def read_wkt(file_path):
    # Step 1: Read the WKT string from the file
    with open(file_path, 'r') as file:
        wkt_string = file.read()
    return wkt_string

In [5]:
#Export MODIS image at 500m with SRS EPSG4326 - same as JRC GSW
to_be_processed_hybas_gt100_id_list = [
    6020017370
]
start_date = '2021-01-01'
end_date = '2022-01-01'
date_format = '%Y-%m-%d'
start_date = datetime.strptime(start_date, date_format)
end_date = datetime.strptime(end_date, date_format)

projection_type = 'Lambert_Azimuthal_Equal_Area'
projection_scale = 500
projection_wkt_path = './projection_wkt/{projection_type}/{hybas_id}.wkt'

for hybas_id in to_be_processed_hybas_gt100_id_list:
    print('Processing HYBAS_ID: {}'.format(hybas_id))
    current_projection_wkt_path = projection_wkt_path.format(projection_type=projection_type, projection_scale=projection_scale, hybas_id=hybas_id)
    current_projection_wkt = read_wkt(current_projection_wkt_path)
    current_date = start_date
    current_region = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_{}".format(hybas_id)).geometry(1).bounds(1)
    while current_date < end_date:
        current_date_str = current_date.strftime(date_format)
        next_date = current_date + relativedelta(months=1)
        next_date_str = next_date.strftime(date_format)
        current_modis_image, Map = ug.image_generation(
            start_time=current_date_str,
            end_time=next_date_str,
            kernel_size=128,
            mode='modis_custom_projection',
            custom_projection_wkt=current_projection_wkt,
            custom_scale=projection_scale,
            input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'],
            input_bands_type='int16'
        )
        #print(current_modis_image.bandTypes().getInfo())
        #for i in range(30):
        #    print(ee.Image(current_modis_image.toList(50).get(i)).projection().getInfo())
        #print(current_modis_image.select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'SensorZenith', 'SolarZenith']).projection().getInfo())
        print(current_modis_image.projection().getInfo())
        task = ee.batch.Export.image.toDrive(
            image=current_modis_image,
            description=f'{hybas_id}_modis_500m_{current_date_str}_{next_date_str}',
            folder='modis_awei_500m_6bands_laea/{}'.format(hybas_id),
            scale=500,
            crs=current_projection_wkt,
            region=current_region,
            maxPixels=1e13
        )
        task.start()
        print(f'{hybas_id}_modis_awei_500m_{current_date_str}_{next_date_str} started')
        current_date = next_date

Processing HYBAS_ID: 2020041390
using regular filter for gsw, start date: 2011-09-01
{'type': 'Projection', 'wkt': 'PROJCS["Lambert Azimuthal Equal Area", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["WGS_1984", \n      SPHEROID["WGS_84", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Lambert_Azimuthal_Equal_Area"], \n  PARAMETER["latitude_of_center", 65.28490342854256], \n  PARAMETER["longitude_of_center", 50.25527266549168], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]', 'transform': [500, 0, 0, 0, -500, 0]}
2020041390_modis_awei_500m_2011-09-01_2011-10-01 started


In [6]:
#Export MODIS image at 500m with SRS EPSG4326 - same as JRC GSW
to_be_processed_hybas_gt100_id_list = [
    3020009320
]

start_date = '2001-01-01'
end_date = '2022-01-01'
date_format = '%Y-%m-%d'
start_date = datetime.strptime(start_date, date_format)
end_date = datetime.strptime(end_date, date_format)

projection_type = 'Lambert_Azimuthal_Equal_Area'
projection_scale = 30
projection_wkt_path = './projection_wkt/{projection_type}/{hybas_id}.wkt'

for hybas_id in to_be_processed_hybas_gt100_id_list:
    print('Processing HYBAS_ID: {}'.format(hybas_id))
    current_projection_wkt_path = projection_wkt_path.format(projection_type=projection_type, projection_scale=projection_scale, hybas_id=hybas_id)
    current_projection_wkt = read_wkt(current_projection_wkt_path)
    current_date = start_date
    current_region = ee.FeatureCollection("projects/xuzhihongqu/assets/HydroLAKES_per_basin_dissolved/hylak_buffered_bounds_dissolved_{}".format(hybas_id)).geometry(1).bounds(1)
    while current_date < end_date:
        current_date_str = current_date.strftime(date_format)
        next_date = current_date + relativedelta(months=1)
        next_date_str = next_date.strftime(date_format)
        current_modis_image, Map = ug.image_generation(
            start_time=current_date_str,
            end_time=next_date_str,
            kernel_size=128,
            mode='gsw_custom_projection',
            custom_projection_wkt=current_projection_wkt,
            custom_scale=projection_scale,
            input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'],
            input_bands_type='int16'
        )
        #print(current_modis_image.bandTypes().getInfo())
        #for i in range(30):
        #    print(ee.Image(current_modis_image.toList(50).get(i)).projection().getInfo())
        #print(current_modis_image.select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'SensorZenith', 'SolarZenith']).projection().getInfo())
        print(current_modis_image.projection().getInfo())
        task = ee.batch.Export.image.toDrive(
            image=current_modis_image,
            description=f'{hybas_id}_gsw_30m_{current_date_str}_{next_date_str}',
            folder='gsw_images_laea_{}'.format(hybas_id),
            scale=30,
            crs=current_projection_wkt,
            region=current_region,
            maxPixels=1e13
        )
        task.start()
        print(f'{hybas_id}_gsw_30m_{current_date_str}_{next_date_str} started')
        current_date = next_date

Processing HYBAS_ID: 3020009320
using regular filter for gsw, start date: 2001-01-01
{'type': 'Projection', 'wkt': 'PROJCS["Lambert Azimuthal Equal Area", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["WGS_1984", \n      SPHEROID["WGS_84", 6378137.0, 298.257223563]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Lambert_Azimuthal_Equal_Area"], \n  PARAMETER["latitude_of_center", 65.20549119504084], \n  PARAMETER["longitude_of_center", 154.99681749560875], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]', 'transform': [30, 0, 0, 0, -30, 0]}
3020009320_gsw_30m_2001-01-01_2001-02-01 started
using regular filter for gsw, start date: 2001-02-01
{'type': 'Projection', 'wkt': 'PROJCS["Lambert Azimuthal Equal Area", \n  GEOGCS["GCS_WGS_1984", \n    DATUM["WGS_1984", \n      SPHEROID["WGS_84", 6378137.0, 2